# Example: Binomial Lattice Simulation of Equity Share Price using Real-World Probabilities
In this example, we will construct a binomial lattice model to simulate future equity share prices using real-world probabilities estimated from historical data.

> __Learning objectives__
> The objective of this example is to familiarize students with applying simple probability mass functions to predict future uncertain quantities such as commodity prices, equity share prices, interest rates, etc. After completing this example, students should be able to:

> * Fill me in later.

Let's get started!
___

## Setup, Data, and Prerequisites
First, we set up the computational environment by including the `Include.jl` file and loading any needed resources.

>__Include:__ The [`include(...)` command](https://docs.julialang.org/en/v1/base/base/#include) evaluates the contents of the input source file, `Include.jl`, in the notebook's global scope. The `Include.jl` file sets paths, loads required external packages, etc. For additional information on functions and types used in this material, see the [Julia programming language documentation](https://docs.julialang.org/en/v1/). 

Let's set up our code environment:

In [3]:
include(joinpath(@__DIR__, "Include.jl")); # include the Include.jl file

    Updating git-repo `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git`
   Resolving package versions...
    Updating `~/Desktop/julia_work/CHEME-5660-CourseRepository-Fall-2025/lectures/week-4/L4b/Project.toml`
  [336ed68f] + CSV v0.10.15
  [c3611d14] + ColorVectorSpace v0.11.0
  [5ae59095] + Colors v0.13.1
  [a93c6f00] + DataFrames v1.8.0
⌅ [864edb3b] + DataStructures v0.18.22
  [31c24e10] + Distributions v0.25.120
  [5789e2e9] + FileIO v1.17.0
  [587475ba] + Flux v0.16.5
  [09f84164] + HypothesisTests v0.11.5
  [916415d5] + Images v0.26.2
⌃ [033835bb] + JLD2 v0.5.15
  [2fda8390] + LsqFit v0.15.1
  [b8f27783] + MathOptInterface v1.44.0
  [0b1bfda6] + OneHotArrays v0.2.10
  [91a5bcdd] + Plots v1.40.20
  [08abe8d2] + PrettyTables v3.0.10
  [10745b16] + Statistics v1.11.1
  [2913bbd2] + StatsBase v0.34.6
  [f3b207a7] + StatsPlots v0.15.7
  [4119e0bf] + VLQuantitativeFinancePackage v1.0.0-DEV `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git#main`
  [ade2

For additional information on functions and types used in this material, see the [Julia programming language documentation](https://docs.julialang.org/en/v1/) and the [VLQuantitativeFinancePackage.jl documentation](https://github.com/varnerlab/VLQuantitativeFinancePackage.jl). 

### Data
We gathered a daily open-high-low-close `dataset` for each firm in the [S&P500](https://en.wikipedia.org/wiki/S%26P_500) from `01-03-2014` until `12-31-2024`, along with data for a few exchange-traded funds and volatility products during that time. 

Let's load the `original_dataset::DataFrame` by calling [the `MyTrainingMarketDataSet()` function](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/data/#VLQuantitativeFinancePackage.MyTrainingMarketDataSet) and remove firms that do not have the maximum number of trading days. The cleaned dataset $\mathcal{D}$ will be stored in the `dataset` variable.

In [6]:
original_dataset = MyTrainingMarketDataSet() |> x-> x["dataset"];

Not all tickers in our dataset have the maximum number of trading days for various reasons, e.g., acquisition or de-listing events. Let's collect only those tickers with the maximum number of trading days.

First, let's compute the number of records for a firm that we know has a maximum value, e.g., `AAPL`, and save that value in the `maximum_number_trading_days::Int64` variable:

In [8]:
maximum_number_trading_days = original_dataset["AAPL"] |> nrow # nrow? (check out: DataFrames.jl)

2767

Now, let's iterate through our data and collect only tickers with `maximum_number_trading_days` records. Save that data in the `dataset::Dict{String,DataFrame}` variable:

In [10]:
dataset = let

    # initialize -
    dataset = Dict{String, DataFrame}();

    # iterate through the dictionary; we can't guarantee a particular order
    for (ticker, data) ∈ original_dataset  # we get each (K, V) pair!
        if (nrow(data) == maximum_number_trading_days) # what is this doing?
            dataset[ticker] = data;
        end
    end
    dataset; # return
end;

How many firms do we have the full number of trading days? Let's use [the `length(...)` method](https://docs.julialang.org/en/v1/base/collections/#Base.length) - notice this works for dictionaries, in addition to arrays, sets and other collections.

In [12]:
length(dataset) # tells us how many keys are in the dictionary (how many firms in our dataset?)

424

Finally, let's get a list of the firms in our cleaned up dataset (and sort them alphabetically). We store the sorted firm ticker symbols in the `list_of_tickers::Array{String,1}` variable.

In [14]:
list_of_tickers = keys(dataset) |> collect |> sort; # list of firm "ticker" symbols in alphabetical order

### Constants
Finally, let's set some constants we'll use later in this notebook. The comments describe the constants, the units, permissible values, etc.

In [16]:
TSIM = 63; # number of trading days to simulate
Δt = (1.0/252); # step size: 1-trading day in units of years

___

## Task 1: Test the lattice model using an example from `Hull`
Let’s start by calculating the hypothetical share price from an example (reproduced from Chapter 13 of Hull) before diving into historical data analysis. The lattice in this example has three levels (`L=0,1,2`), with an initial share price of $S_{\circ}=20.0$ USD. An `up` move has a probability of `p = 0.6523` and a magnitude of `u = 1.1`, while a `down` move has a magnitude of `d = 0.9`.

Calculating the future share price with a binomial lattice requires setting the model’s parameters, including the initial share price (as a `Float64`), the number of time steps to simulate into the future, which equals the number levels of the tree starting from zero (as an `Int64`), the up and down move magnitudes (as `Float64` values), and the probability of an up move (as a `Float64`):

In [19]:
Sₒ,T,u,d,p = 20.0,2,1.1,0.9,0.6523;

Once these values are set, we use the `build(…)` function to create an empty lattice model of [type `MyBinomialEquityPriceTree`](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/equity/#VLQuantitativeFinancePackage.MyBinomialEquityPriceTree), which is then passed to [the `populate(…)` function](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/equity/#VLQuantitativeFinancePackage.populate-Tuple{MyBinomialEquityPriceTree}) using the [Julia piping `|>` operator](https://docs.julialang.org/en/v1/manual/functions/#Function-composition-and-piping). 

The [`populate(…)` function](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/equity/#VLQuantitativeFinancePackage.populate-Tuple{MyBinomialEquityPriceTree}) calculates the prices and probabilities of each node, which are of [type `MyBiomialLatticeEquityNodeModel`](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/equity/#VLQuantitativeFinancePackage.MyBiomialLatticeEquityNodeModel), in the tree:

In [21]:
test_hull_model = build(MyBinomialEquityPriceTree, (
        u = u, d = d, p = p)) |> (x-> populate(x, Sₒ = Sₒ, h = T));

### Check: Do we reproduce the `Hull` tree values?
Finally, we can compare the price values we calculated to those reported by `Hull.` First, we populate the `hull_price_dictionary` dictionary, which holds `key=>value` pairs encoding the data for each node in the tree. In this dictionary, the `keys` are the node indices, while the `values` in the dictionary are the price values at that node:

In [23]:
hull_price_dictionary = Dict(0=>20.0, 1=>22.0,2=>18.0,3=>24.2,4=>19.8,5=>16.2);

Now, let's populate `hull_test_data_table,` which is of [type `DataFrame`](https://dataframes.juliadata.org/stable/), and display the contents using [the `pretty_table(...)` method, exported by PrettyTable.jl package](https://github.com/ronisbr/PrettyTables.jl) using a `for` loop. For iteration of the `for` loop:
* We capture the node index `i`, the share price from `Hull`, and our computed price at that node from the binomial tree
* We use the [isapprox function](https://docs.julialang.org/en/v1/base/math/#Base.isapprox), with a relative tolerance `rtol = 1e-4`, to check if our computed price aggress with the price reported by `Hull.` If yes, a value of `true` is reported, otherwise `false`

In [25]:
number_of_nodes = length(test_hull_model.data);
hull_test_data_table = DataFrame();
for i ∈ 0:(number_of_nodes-1)
    
    row_data = (
        node = i,
        hull_price = hull_price_dictionary[i],
        our_price =  test_hull_model.data[i].price, 
        isapproxequal = isapprox(hull_price_dictionary[i], test_hull_model.data[i].price, rtol=1e-4)
    );
    
    push!(hull_test_data_table, row_data)
end
pretty_table(hull_test_data_table, tf=tf_simple)

LoadError: UndefVarError: `tf_simple` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

___

## Task 2: Estimate the `up,` `down,` and probability of an up move `p` for a random firm, and simulate the future share price
After validating our lattice implementation, it's time to utilize historical data to compute a share price prediction. To create a binomial lattice model for future share prices, we must estimate three critical parameters: $p$, $u$, and $d$.

* The $p$ parameter represents the probability of a share price increase or an `up` move between two periods $j\rightarrow{j+1}$. As a binary lattice model only allows `up` and `down` moves, the probability of a `down` move is $1-p$.
* The $u$ parameter represents the amount of an `up` move. If $S_{j}$ stands for the share price in period $j$, and $S_{j+1}$ is the share price in the next period, then an `up` move will give $S_{j+1} = u\cdot{S}_{j}$.
* The $d$ parameter represents the amount of a `down` move. If $S_{j}$ stands for the share price in period $j$, and $S_{j+1}$ is the share price in the next period, then a `down` move will give $S_{j+1} = d\cdot{S}_{j}$.

To start, let's select a random firm from the dataset:

In [28]:
# random_firm_ticker = rand(list_of_all_tickers);
random_firm_ticker = "SPY"
random_firm_index = findfirst(x-> x == random_firm_ticker, list_of_all_tickers);
random_firm_data = dataset[random_firm_ticker];

LoadError: UndefVarError: `list_of_all_tickers` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

### Estimate the u, d, and probability p parameters from the data
Let's estimate the up $u$, down $d$ factors and the probability $p$ from historical data.

__Initialize__: Given the growth rate sequence $\{\mu_{2},\mu_{3},\dots,\mu_{T}\}$ for firm $(i)$ (we neglect the superscript $i$ for simplicity) and a time step $\Delta{t} > 0$ (units: years). Initialize the up factors collection $U = \emptyset$ and down factors collection $D = \emptyset$.

1. For $t = 2,3,\dots,T$ __do__:
    - If $\mu_{t} > 0$, then update $U \gets U \cup \{e^{\mu_{t}\Delta{t}}\}$ where $U$ is a collection of up factors.
    - If $\mu_{t} < 0$, then update $D \gets D \cup \{e^{\mu_{t}\Delta{t}}\}$ where $D$ is a collection of down factors.
    - If $\mu_{t} = 0$, skip (no price change).
2. Compute the up factor $u$ as the mean of the up factors collection $U$: $u = \frac{1}{|U|} \sum_{v \in U} v$.
3. Compute the down factor $d$ as the mean of the down factors collection $D$: $d = \frac{1}{|D|} \sum_{v \in D} v$.
4. Compute the probability $p$ as the fraction of up movements: $p = \frac{|U|}{|U| + |D|}$.


We use daily data; thus, the natural time frame between $S_{j-1}$ and $S_{j}$ is a single day. However, it will be easier to use an annualized value for the $\mu$ parameter; thus, we let $\Delta{t} = 1/252$, i.e., the fraction of a year that occurs in a single trading day.

The [`log_growth_matrix(...)` method](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/equity/#VLQuantitativeFinancePackage.log_growth_matrix) takes the cleaned dataset $\mathcal{D}$, which contains $T$ days of data for each firm, a list of firms $\mathcal{L}$ and computes the growth rate values for each firm as a function of time. 

The data is returned as a $(T - 1)\times\dim\mathcal{L}$ array (time on the rows, firm $i$ on the columns). We store the data in the `log_growth_array::Array{Float64,1}` variable:

In [30]:
log_growth_array = log_growth_matrix(dataset, random_firm_ticker); # array holding growth rate time series

Using the `log_growth_array::Array{Float64,1},` we compute the expected magnitude of an `up` move `ū,` the expected magnitude of a `down` move `d̄,` and the estimated probability `p̄` of an `up` move (assuming a real-world probability measure) by passing the return data to [a `RealWorldBinomialProbabilityMeasure` instance](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/equity/#VLQuantitativeFinancePackage.RealWorldBinomialProbabilityMeasure), which returns the $(u,d,p)$-tuple

> __Syntax sugar__: behind the scenes, this call points to a private function that computes the `ū,` `d̄,` and `p̄` by analyzing the growth rate time series data in the `log_growth_array::Array{Float64,1}` variable. It looks we are using a type as a function name, but we are not. This is a common pattern in Julia programming.

In [32]:
(ū,d̄,p̄) = (RealWorldBinomialProbabilityMeasure())(log_growth_array; Δt = (1.0/252.0))

(1.005564502936516, 0.9938360721560962, 0.564352856109906)

### TODO: Build binomial lattice model using historical $(u,d,p)$ parameters, simulate future prices
Let's construct an instance of [the `MyBinomialEquityPriceTree` type](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/equity/#VLQuantitativeFinancePackage.MyBinomialEquityPriceTree) using the estimated values for `(u,d,p).` This enables us to calculate the prices and probabilities in the tree. We store the populated model in the variable `model` for future use.

* Firstly, we must set a value for the `T` variable, specifying the number of future days to simulate. Time steps are represented in the tree as levels `L = 0,1,..., T`, where each level represents $\Delta{t}$ units of time, which is a trading day. We then specify the `start_index` as the trading day index in the combined dataset `df,` which will serve as the tree’s starting point or `L=0`. Finally, we can set the variable `Sₒ,` which corresponds to the initial price per share at the root of the tree; we use the [volume-weighted average price (VWAP)](https://en.wikipedia.org/wiki/Volume-weighted_average_price) as the initial condition:

In [34]:
# 1277 - 1340
start_index = rand(1:(maximum_number_trading_days - TSIM - 1))
stop_index = start_index + TSIM
println("Visualize Firm-$(random_firm_index) between trading days ($(start_index) -> $(stop_index))")

LoadError: UndefVarError: `random_firm_index` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [35]:
Sₒ = random_firm_data[start_index,:volume_weighted_average_price];
random_test_model = build(MyBinomialEquityPriceTree, (
    u = ū, d = d̄, p = p̄)) |> (model -> populate(model, Sₒ = Sₒ, h = TSIM));
println("Starting simulation from Sₒ = $(Sₒ) USD/share")

LoadError: UndefVarError: `random_firm_data` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

#### Check: Visualize price distribution at tree level `L`

In [37]:
let
    L = TSIM;
    price_array = random_test_model.levels[L] .|> x-> random_test_model.data[x].price
    probability_array = random_test_model.levels[L] .|> x-> random_test_model.data[x].probability;
    scatter(price_array, probability_array, label="Firm-$(random_firm_ticker) at L = $(L)")
    xlabel!("Share price Firm-$(random_firm_ticker) (USD)", fontsize=18)
    ylabel!("Probability P(X=k)", fontsize=18)
end

LoadError: UndefVarError: `random_test_model` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

### Visualize binomial lattice simulation
Next, look at how the binomial model simulations compare with market observations. In particular, we'll compute the expected value and the variance of the simulated price as a function of time, i.e., at each level in the tree using the local [`𝔼(...)`](src/Compute.jl) and [`Var(...)` functions](src/Compute.jl).
* The `levels` array holds the index of the tree levels, i.e., the time slices. The [`𝔼(...)`](src/Compute.jl) and [`Var(...)` functions](src/Compute.jl) take the `random_test_model` tree model, and the `levels` array and return the expectation and the variance, respectively. 

In [39]:
levels = [k for k ∈ 0:(TSIM-1)]
expectation = 𝔼(random_test_model, levels); # compute the expected value
variance = Var(random_test_model, levels); # compute the variance

LoadError: UndefVarError: `𝔼` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

#### Summary
`Unhide` the code block below to see how we plotted the model simulations and the observed prices for a randomly selected firm using a binomial lattice model.
* Depending upon the random ticker, the observed price (red line) may or may not lie in the $\pm\,{z}\cdot\sigma$ confidence ranges, where `z = {1,1.96,2.576}` which are shown in progressively lighter shades of blue. The dashed dark blue line shows the expected value.

In [41]:
let 
    L68 = expectation[:,2] .- sqrt.(variance[:,2])
    U68 = expectation[:,2] .+ sqrt.(variance[:,2])
    L95 = expectation[:,2] .- 1.96*sqrt.(variance[:,2])
    U95 = expectation[:,2] .+ 1.96*sqrt.(variance[:,2])
    L99 = expectation[:,2] .- 2.576*sqrt.(variance[:,2])
    U99 = expectation[:,2] .+ 2.576*sqrt.(variance[:,2])

    T_range = range(start_index,stop=stop_index-1,step=1) |> collect 
    T_sim = T_range*Δt .- Δt

    plot(T_sim, expectation[:,2], fillrange=L68, c=:deepskyblue1, alpha=0.4, label="")
    plot!(T_sim, expectation[:,2], fillrange=U68, c=:deepskyblue1, alpha=0.4, label="")
    plot!(T_sim, expectation[:,2], fillrange=L95, c=:deepskyblue1, alpha=0.3, label="")
    plot!(T_sim, expectation[:,2], fillrange=U95, c=:deepskyblue1, alpha=0.3, label="")
    plot!(T_sim, expectation[:,2], fillrange=L99, c=:deepskyblue1, alpha=0.2, label="")
    plot!(T_sim, expectation[:,2], fillrange=U99, c=:deepskyblue1, alpha=0.2, label="")
    plot!(T_sim, expectation[:,2],c=:blue, lw=3,ls=:dash, label="expectation")
    plot!(T_sim, random_firm_data[levels .+ start_index, :volume_weighted_average_price],  
        lw=4, c=:red, label="Firm-$(random_firm_ticker)", legend = :topleft)
    scatter!(T_sim, random_firm_data[levels .+ start_index, :volume_weighted_average_price], label="", 
        msc=:red, mc=:white)

    xlabel!("Time (years)", fontsize=18)
    ylabel!("Firm-$(random_firm_ticker) VWAP (USD/share)", fontsize=18)
end

LoadError: MethodError: no method matching getindex(::typeof(expectation), ::Colon, ::Int64)
The function `getindex` exists, but no method is defined for this combination of argument types.

The black dashed line is the expected share price predicted by the lattice model, while the shaded regions denote $\mu\pm{z}\cdot\sigma$ for $z=$`1.0,1.96,2.576`. The solid red line with circles is the [observed VWAP price](https://en.wikipedia.org/wiki/Volume-weighted_average_price) for this firm, and date range.

### Check: Was this a `successful` simulation?
Determining if a model accurately represents data can be done in various ways, but we will utilize a simple, straightforward, functional test. This test involves checking if the simulated price falls within a specified lower bound ($L_{j}$) and upper bound ($U_{j}$) for all times $t_j$ within a given interval ($t_{j}\in\mathcal{I}_{k}$). 
* __Success__: If the simulation meets this requirement, it is deemed a `success.` The values for $L_{j}$ and $U_{j}$ can be customized, but we set these to $\mathbb{E}(X)\pm{2.576}\cdot{\sigma}$, where $\mathbb{E}(X)$ represents the expected value and $\sigma$ is the standard deviation of the binomial lattice model simulation. 
* __Failure__: However, if the actual price exceeds the given bounds at any point, the simulation is considered a `failure.`

In [44]:
let
    z = 2.576
    L = expectation[:,2] .- z*sqrt.(variance[:,2])
    U = expectation[:,2] .+ z*sqrt.(variance[:,2])
    
    # check the bounds: if below L, or above U then simulation failed
    success_flag = true;
    for k ∈ 1:TSIM
        
        price_value = random_firm_data[start_index+k-1,:volume_weighted_average_price];    
        if (price_value < L[k]) || (price_value > U[k])            
            success_flag = false;
            break;
        end
    end

    println("The simultion of the VWAP of Firm-$(random_firm_index) between ($(start_index) -> $(stop_index)) was successful: $(success_flag)")
end

LoadError: MethodError: no method matching getindex(::typeof(expectation), ::Colon, ::Int64)
The function `getindex` exists, but no method is defined for this combination of argument types.

## Disclaimer and Risks
__This content is offered solely for training and informational purposes__. No offer or solicitation to buy or sell securities or derivative products or any investment or trading advice or strategy is made, given, or endorsed by the teaching team. 

__Trading involves risk__. Carefully review your financial situation before investing in securities, futures contracts, options, or commodity interests. Past performance, whether actual or indicated by historical tests of strategies, is no guarantee of future performance or success. Trading is generally inappropriate for someone with limited resources, investment or trading experience, or a low-risk tolerance.  Only risk capital that is not required for living expenses.

__You are fully responsible for any investment or trading decisions you make__. Such decisions should be based solely on evaluating your financial circumstances, investment or trading objectives, risk tolerance, and liquidity needs.